In [5]:
import re
# Five letters, not a proper noun
wordle_word = re.compile(r"^[a-z]....$")

In [7]:
import urllib.request
with urllib.request.urlopen("https://web.mit.edu/freebsd/head/share/dict/web2") as w:
  words = [word for word in w.read().decode("utf-8").splitlines() if wordle_word.match(word)]

In [9]:
import random
i = random.randrange(len(words))
correct = words[i]
print(f"{i}: {correct}")

4085: liege


In [10]:
def wordle_round(guess, correct):
    result = ""
    for i in range(5):
        if guess[i] == correct[i]:
            result = result + "+"
        elif guess[i] in correct:
            result = result + "o"
        else:
            result = result + "-"
    return result

In [11]:
def process_result(guess, result, exact, exactnt, must, mustnt):
    for i in range(5):
        if result[i] == "+":
            exact[i] = guess[i]
            must.add(guess[i])
        elif result[i] == "o":
            exactnt[i].add(guess[i])
            must.add(guess[i])
        else:
            mustnt.add(guess[i])
    return exact, exactnt, must, mustnt

In [24]:
def next_guess(exact, exactnt, must, mustnt, guesses):
    exactly = re.compile("^" + "".join(exact) + "$")
    exactntly = re.compile("^" + "".join("[^" + "".join(letters) + "]" if letters else "." for letters in exactnt) + "$")
    filtered_words = [
        word
        for word in words
        if word not in guesses and \
            exactly.match(word) and \
            exactntly.match(word) and \
            all(letter in word for letter in must) and \
            not any(letter in word for letter in mustnt)
    ]
    scores = { word: [0, 0] for word in filtered_words }
    for test in filtered_words:
        for target in filtered_words:
            if test == target:
                continue
            if any(letter in target for letter in test if letter not in must):
                scores[test][0] += len(set(test).intersection(set(target)))
            else:
                scores[test][1] += 1
    max_score = -1
    guess = ""
    remaining = len(filtered_words)
    matching = set()
    for word, score in scores.items():
        # Round 1 of scoring: prefer halving the solution space
        adjusted_score = remaining/2 - abs(remaining/2 - score[1])
        if adjusted_score > max_score:
            max_score = adjusted_score
            guess = word
            matching=set([word])
        elif adjusted_score == max_score:
            matching.add(word)
    max_score = -1
    for word in matching:
        # Round 2 of scoring: prefer more letter overlap
        adjusted_score = scores[word][0]
        if adjusted_score > max_score:
            max_score = adjusted_score
            guess = word
    print(matching)
    print(f"{scores[guess]}")
    return guess, remaining

In [13]:
def starting_word():
    # Run this when changing next_guess to re-compute an optimal initial guess
    exact = ["."] * 5
    exactnt = [set()] * 5
    must = set()
    mustnt = set()
    guesses = []
    guess, outof = next_guess(exact, exactnt, must, mustnt, guesses)
    print(f"Initial guess: {guess} out of {outof}")

In [20]:
def play(correct):
    #guess = "ourie"
    guess = "irate"
    exact = ["."] * 5
    exactnt = [set(), set(), set(), set(), set()] # can't use [set()] * 5
    must = set()
    mustnt = set()
    guesses = []
    r = 1
    while r < 100:
        print(f"round {r}:")
        print(f"{guess}")
        result = wordle_round(guess, correct)
        print(f"{result}")
        if result == "+++++":
            break
        r += 1
        guesses.append(guess)
        exact, exactnt, must, mustnt = process_result(guess, result, exact, exactnt, must, mustnt)
        print("required   letters: " + "".join(must))
        print("prohibited letters: " + "".join(mustnt))
        print("required   pattern: " + "".join(exact))
        print("prohibited pattern: " + ", ".join("".join(letters) for letters in exactnt))
        guess, outof = next_guess(exact, exactnt, must, mustnt, guesses)
        print(f"{outof} words remaining")
        print()
    return r

In [25]:
play(correct)

round 1:
irate
o---+
required   letters: ie
prohibited letters: rta
required   pattern: ....e
prohibited pattern: i, , , , 
{'shice', 'josie', 'folie', 'dwine'}
[255, 80]
160 words remaining

round 2:
shice
--o-+
required   letters: ie
prohibited letters: rhcsta
required   pattern: ....e
prohibited pattern: i, , i, , 
{'diene', 'midge', 'binge', 'didle', 'fidge', 'liege'}
[91, 27]
55 words remaining

round 3:
binge
-+-++
required   letters: ieg
prohibited letters: rnhbcsta
required   pattern: .i.ge
prohibited pattern: i, , i, , 
{'liege', 'midge', 'fidge'}
[4, 1]
3 words remaining

round 4:
midge
-+-++
required   letters: ieg
prohibited letters: rnmhbcdsta
required   pattern: .i.ge
prohibited pattern: i, , i, , 
{'liege'}
[0, 0]
1 words remaining

round 5:
liege
+++++


5

In [ ]:
play("tabet")

round 1: irate
         --ooo
required   letters: aet
prohibited letters: ir
required   pattern: .....
prohibited pattern: , , a, t, e
98 words remaining

round 2: ental
         o-oo-
required   letters: aet
prohibited letters: ilrn
required   pattern: .....
prohibited pattern: e, , at, at, e
18 words remaining

round 3: ashet
         o--++
required   letters: aet
prohibited letters: rnislh
required   pattern: ...et
prohibited pattern: ae, , at, at, e
4 words remaining

round 4: cadet
         -+-++
required   letters: aet
prohibited letters: rncdislh
required   pattern: .a.et
prohibited pattern: ae, , at, at, e
2 words remaining

round 5: tapet
         ++-++
required   letters: aet
prohibited letters: rncdpislh
required   pattern: ta.et
prohibited pattern: ae, , at, at, e
1 words remaining

round 6: tabet
         +++++


6

In [ ]:
guess = "irate"
exact = ["."] * 5
exactnt = [set(), set(), set(), set(), set()] # can't use [set()] * 5
must = set()
mustnt = set()
guesses = [ "ental" ]
r = 1
result = "--ooo"
guesses.append(guess)
exact, exactnt, must, mustnt = process_result(guess, result, exact, exactnt, must, mustnt)
print("required   letters: " + "".join(must))
print("prohibited letters: " + "".join(mustnt))
print("required   pattern: " + "".join(exact))
print("prohibited pattern: " + ", ".join("".join(letters) for letters in exactnt))
guess, outof = next_guess(exact, exactnt, must, mustnt, guesses)
print(guess)
print(f"{outof} words remaining")


required   letters: aet
prohibited letters: ir
required   pattern: .....
prohibited pattern: , , a, t, e
laten
97 words remaining


In [ ]:
result = "oooo-"
guesses.append(guess)
exact, exactnt, must, mustnt = process_result(guess, result, exact, exactnt, must, mustnt)
print("required   letters: " + "".join(must))
print("prohibited letters: " + "".join(mustnt))
print("required   pattern: " + "".join(exact))
print("prohibited pattern: " + ", ".join("".join(letters) for letters in exactnt))
guess, outof = next_guess(exact, exactnt, must, mustnt, guesses)
print(guess)
print(f"{outof} words remaining")


required   letters: alet
prohibited letters: nir
required   pattern: .....
prohibited pattern: l, a, at, et, e
pleat
10 words remaining


In [ ]:
result = "++++-"
guesses.append(guess)
exact, exactnt, must, mustnt = process_result(guess, result, exact, exactnt, must, mustnt)
print("required   letters: " + "".join(must))
print("prohibited letters: " + "".join(mustnt))
print("required   pattern: " + "".join(exact))
print("prohibited pattern: " + ", ".join("".join(letters) for letters in exactnt))
guess, outof = next_guess(exact, exactnt, must, mustnt, guesses)
print(guess)
print(f"{outof} words remaining")

required   letters: hars
prohibited letters: tepiouyk
required   pattern: shar.
prohibited pattern: , a, r, , r
shard
2 words remaining


In [ ]:
hist = [0] * 20
for i in range(100):
    rounds = play(words[random.randrange(len(words))])
    hist[rounds] += 1
print(hist)

round 1: irate
         o--oo
required   letters: ite
prohibited letters: ar
required   pattern: .....
prohibited pattern: i, , , t, e
52 words remaining

round 2: steid
         oooo-
required   letters: ites
prohibited letters: ard
required   pattern: .....
prohibited pattern: is, t, e, it, e
5 words remaining

round 3: zeist
         -++++
required   letters: ites
prohibited letters: ardz
required   pattern: .eist
prohibited pattern: is, t, e, it, e
2 words remaining

round 4: neist
         +++++
round 1: irate
         -o+--
required   letters: ar
prohibited letters: ite
required   pattern: ..a..
prohibited pattern: , r, , , 
65 words remaining

round 2: scaur
         +-+-o
required   letters: ars
prohibited letters: teciu
required   pattern: s.a..
prohibited pattern: , r, , , r
15 words remaining

round 3: sharp
         +-++-
required   letters: ars
prohibited letters: tepciuh
required   pattern: s.ar.
prohibited pattern: , r, , , r
8 words remaining

round 4: swarm
         ++

2 words remaining

round 6: paper
         +++++
round 1: irate
         --o--
required   letters: a
prohibited letters: iter
required   pattern: .....
prohibited pattern: , , a, , 
768 words remaining

round 2: onlay
         o-+o-
required   letters: lao
prohibited letters: terniy
required   pattern: ..l..
prohibited pattern: o, , a, a, 
18 words remaining

round 3: aslop
         o-+o-
required   letters: lao
prohibited letters: ternpiys
required   pattern: ..l..
prohibited pattern: ao, , a, ao, 
5 words remaining

round 4: callo
         -o+oo
required   letters: lao
prohibited letters: ternpciys
required   pattern: ..l..
prohibited pattern: ao, a, a, lao, o
2 words remaining

round 5: molka
         -++-+
required   letters: lao
prohibited letters: ternpciysmk
required   pattern: .ol.a
prohibited pattern: ao, a, a, lao, o
1 words remaining

round 6: volva
         +++++
round 1: irate
         o-+--
required   letters: ia
prohibited letters: ter
required   pattern: ..a..
prohibite

603 words remaining

round 2: souly
         oo-o-
required   letters: los
prohibited letters: teriayu
required   pattern: .....
prohibited pattern: s, o, , l, 
6 words remaining

round 3: klosh
         -++++
required   letters: hlos
prohibited letters: teriayuk
required   pattern: .losh
prohibited pattern: s, o, , l, 
2 words remaining

round 4: closh
         +++++
round 1: irate
         -----
required   letters: 
prohibited letters: teria
required   pattern: .....
prohibited pattern: , , , , 
603 words remaining

round 2: souly
         +---o
required   letters: ys
prohibited letters: teriaoul
required   pattern: s....
prohibited pattern: , , , , y
1 words remaining

round 3: synch
         +++++
round 1: irate
         -oooo
required   letters: aert
prohibited letters: i
required   pattern: .....
prohibited pattern: , r, a, t, e
49 words remaining

round 2: ratel
         oooo-
required   letters: aert
prohibited letters: il
required   pattern: .....
prohibited pattern: r, ar, at

320 words remaining

round 2: loser
         +--o+
required   letters: ler
prohibited letters: tiaos
required   pattern: l...r
prohibited pattern: , r, , e, e
1 words remaining

round 3: lemur
         +++++
round 1: irate
         o-oo-
required   letters: iat
prohibited letters: er
required   pattern: .....
prohibited pattern: i, , a, t, 
73 words remaining

round 2: altin
         +-o+-
required   letters: iat
prohibited letters: lern
required   pattern: a..i.
prohibited pattern: i, , at, t, 
5 words remaining

round 3: atmid
         +o-+-
required   letters: iat
prohibited letters: erndml
required   pattern: a..i.
prohibited pattern: i, t, at, t, 
1 words remaining

round 4: absit
         +++++
round 1: irate
         ---o-
required   letters: t
prohibited letters: iaer
required   pattern: .....
prohibited pattern: , , , t, 
153 words remaining

round 2: shout
         --o-o
required   letters: to
prohibited letters: eriasuh
required   pattern: .....
prohibited pattern: , , o, t,

35 words remaining

round 2: crout
         +++++
round 1: irate
         o----
required   letters: i
prohibited letters: aert
required   pattern: .....
prohibited pattern: i, , , , 
456 words remaining

round 2: lysin
         -o-o-
required   letters: iy
prohibited letters: ternasl
required   pattern: .....
prohibited pattern: i, y, , i, 
33 words remaining

round 3: midgy
         -+-o+
required   letters: igy
prohibited letters: terndamsl
required   pattern: .i..y
prohibited pattern: i, y, , ig, 
1 words remaining

round 4: gibby
         +++++
round 1: irate
         -o+o-
required   letters: art
prohibited letters: ie
required   pattern: ..a..
prohibited pattern: , r, , t, 
22 words remaining

round 2: scart
         +-+++
required   letters: arst
prohibited letters: iec
required   pattern: s.art
prohibited pattern: , r, , t, 
6 words remaining

round 3: swart
         +-+++
required   letters: arst
prohibited letters: iewc
required   pattern: s.art
prohibited pattern: , r, , t, 

6 words remaining

round 4: pyxie
         --+++
required   letters: iex
prohibited letters: trndwpaosyl
required   pattern: ..xie
prohibited pattern: i, , i, , 
1 words remaining

round 5: jixie
         +++++
round 1: irate
         -oooo
required   letters: aert
prohibited letters: i
required   pattern: .....
prohibited pattern: , r, a, t, e
49 words remaining

round 2: ratel
         +o+o-
required   letters: aert
prohibited letters: il
required   pattern: r.t..
prohibited pattern: , ar, a, te, e
3 words remaining

round 3: retan
         +++++
round 1: irate
         o---+
required   letters: ie
prohibited letters: art
required   pattern: ....e
prohibited pattern: i, , , , 
160 words remaining

round 2: sline
         o-+o+
required   letters: iens
prohibited letters: lart
required   pattern: ..i.e
prohibited pattern: is, , , n, 
1 words remaining

round 3: noise
         +++++
round 1: irate
         -+-o-
required   letters: tr
prohibited letters: iae
required   pattern: .r...
p

768 words remaining

round 2: onlay
         --oo+
required   letters: lay
prohibited letters: ternio
required   pattern: ....y
prohibited pattern: , , la, a, 
12 words remaining

round 3: amply
         +-+++
required   letters: lapy
prohibited letters: terniom
required   pattern: a.ply
prohibited pattern: , , la, a, 
1 words remaining

round 4: apply
         +++++
round 1: irate
         ---o-
required   letters: t
prohibited letters: iaer
required   pattern: .....
prohibited pattern: , , , t, 
153 words remaining

round 2: shout
         -oo-o
required   letters: hto
prohibited letters: eriasu
required   pattern: .....
prohibited pattern: , h, o, t, t
11 words remaining

round 3: cothy
         o++o-
required   letters: htoc
prohibited letters: eriaysu
required   pattern: .ot..
prohibited pattern: c, h, o, ht, t
6 words remaining

round 4: gotch
         +++++
round 1: irate
         ooo-o
required   letters: iaer
prohibited letters: t
required   pattern: .....
prohibited pattern: 

In [ ]:
hist

[0, 0, 2, 24, 44, 18, 8, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
hist[1:10]

[0, 2, 24, 44, 18, 8, 1, 0, 3]